In [1]:
import pongGym
import random
import numpy as np
import os
env = pongGym.DoublePong()
PAD_HEIGHT = 80
HALF_PAD_HEIGHT = PAD_HEIGHT // 2

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def encode(state):
    pd1y = state[21]
    pd2y = state[23]
    st = [(max(0, state[i*4+0] + state[i*4+2] * 6), abs(state[i*4+1] + state[i*4+3] * 6), state[i*4+2], state[i*4+3]) for i in range(5)]
    st.sort(key = lambda s: s[0])

    now = 1
    re = 0
#     print(st)
    for i in range(5):
        ty = 0
        if(st[i][1] > pd1y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd1y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
        ty = 0
        if(st[i][1] > pd2y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd2y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
    return re
state = env.reset()

In [3]:

q_table = np.zeros([(3*3)**5, env.action_space.n])
weight_path = "q_learning_{}.npy".format(6)
if os.path.isfile(weight_path):
    with open(weight_path, 'rb') as f:
        q_table = np.load(f)

In [4]:
print(q_table)

[[ 0.1274877   0.07788784  0.07879286  0.07786862]
 [-0.30133649  0.19674236 -0.44835252 -0.50409405]
 [-0.16218652 -0.33833229  0.04829282  0.32232379]
 ...
 [ 0.17429188  1.20161317  0.18021038  0.19645067]
 [-0.71974425 -0.33437606 -1.00515132 -0.80647568]
 [-0.29368287 -0.66451678 -0.58727237 -0.00464195]]


In [5]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.2

# For plotting metrics
all_epochs = []
all_penalties = []
all_reward = 0

for i in range(1, 100001):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        next_state, reward, done, info = env.step(action)
        next_state = encode(next_state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
        all_reward += reward
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
#         clear_output(wait=True)
        print(f"Episode: {i}")
        print(f"average reward: {all_reward / 100}")
        all_reward = 0

print("Training finished.\n")

Episode: 100
average reward: -91.25
Episode: 200
average reward: -53.9
Episode: 300
average reward: -20.46
Episode: 400
average reward: -29.71
Episode: 500
average reward: 20.61
Episode: 600
average reward: 11.73
Episode: 700
average reward: 46.59
Episode: 800
average reward: 23.29
Episode: 900
average reward: -13.98
Episode: 1000
average reward: -31.06
Episode: 1100
average reward: -36.25
Episode: 1200
average reward: 47.82
Episode: 1300
average reward: -24.63
Episode: 1400
average reward: -2.0
Episode: 1500
average reward: -15.62
Episode: 1600
average reward: 46.31
Episode: 1700
average reward: 7.57
Episode: 1800
average reward: -40.66
Episode: 1900
average reward: -40.24
Episode: 2000
average reward: -39.01
Episode: 2100
average reward: 30.51
Episode: 2200
average reward: 52.61
Episode: 2300
average reward: -26.0
Episode: 2400
average reward: -19.59
Episode: 2500
average reward: -20.8
Episode: 2600
average reward: -17.94
Episode: 2700
average reward: -1.23
Episode: 2800
average rewa

Episode: 22400
average reward: 30.44
Episode: 22500
average reward: 21.19
Episode: 22600
average reward: 49.57
Episode: 22700
average reward: -12.0
Episode: 22800
average reward: -16.1
Episode: 22900
average reward: -40.37
Episode: 23000
average reward: 61.56
Episode: 23100
average reward: 91.12
Episode: 23200
average reward: 53.92
Episode: 23300
average reward: 9.08
Episode: 23400
average reward: 2.42
Episode: 23500
average reward: -45.23
Episode: 23600
average reward: -35.71
Episode: 23700
average reward: 62.46
Episode: 23800
average reward: -16.46
Episode: 23900
average reward: -5.58
Episode: 24000
average reward: -45.86
Episode: 24100
average reward: -18.3
Episode: 24200
average reward: -6.6
Episode: 24300
average reward: -20.69
Episode: 24400
average reward: -58.51
Episode: 24500
average reward: 0.42
Episode: 24600
average reward: -37.78
Episode: 24700
average reward: -2.32
Episode: 24800
average reward: -16.53
Episode: 24900
average reward: -6.84
Episode: 25000
average reward: 2.

Episode: 44500
average reward: -59.58
Episode: 44600
average reward: -59.33
Episode: 44700
average reward: -49.27
Episode: 44800
average reward: -31.11
Episode: 44900
average reward: -62.63
Episode: 45000
average reward: -26.13
Episode: 45100
average reward: 37.3
Episode: 45200
average reward: 25.75
Episode: 45300
average reward: -47.29
Episode: 45400
average reward: -1.55
Episode: 45500
average reward: 9.83
Episode: 45600
average reward: -5.28
Episode: 45700
average reward: -68.63
Episode: 45800
average reward: -30.89
Episode: 45900
average reward: -9.99
Episode: 46000
average reward: -17.65
Episode: 46100
average reward: -28.27
Episode: 46200
average reward: 18.54
Episode: 46300
average reward: -52.08
Episode: 46400
average reward: -64.92
Episode: 46500
average reward: -10.32
Episode: 46600
average reward: 19.9
Episode: 46700
average reward: -15.96
Episode: 46800
average reward: -6.37
Episode: 46900
average reward: -51.87
Episode: 47000
average reward: 39.73
Episode: 47100
average re

Episode: 66500
average reward: 35.4
Episode: 66600
average reward: 67.62
Episode: 66700
average reward: 39.09
Episode: 66800
average reward: -23.88
Episode: 66900
average reward: -42.71
Episode: 67000
average reward: -32.83
Episode: 67100
average reward: 4.93
Episode: 67200
average reward: 0.61
Episode: 67300
average reward: -10.34
Episode: 67400
average reward: -8.34
Episode: 67500
average reward: -40.46
Episode: 67600
average reward: -32.1
Episode: 67700
average reward: 5.99
Episode: 67800
average reward: 35.94
Episode: 67900
average reward: 19.66
Episode: 68000
average reward: -24.87
Episode: 68100
average reward: -13.44
Episode: 68200
average reward: -65.9
Episode: 68300
average reward: -33.18
Episode: 68400
average reward: 25.31
Episode: 68500
average reward: -11.19
Episode: 68600
average reward: -10.64
Episode: 68700
average reward: -80.87
Episode: 68800
average reward: 3.08
Episode: 68900
average reward: -53.58
Episode: 69000
average reward: -75.7
Episode: 69100
average reward: 

Episode: 88500
average reward: -8.07
Episode: 88600
average reward: -60.36
Episode: 88700
average reward: -19.05
Episode: 88800
average reward: -48.91
Episode: 88900
average reward: -20.18
Episode: 89000
average reward: -67.0
Episode: 89100
average reward: 8.52
Episode: 89200
average reward: -71.75
Episode: 89300
average reward: -48.27
Episode: 89400
average reward: -25.41
Episode: 89500
average reward: -13.67
Episode: 89600
average reward: -16.29
Episode: 89700
average reward: 40.69
Episode: 89800
average reward: 10.53
Episode: 89900
average reward: 3.85
Episode: 90000
average reward: -23.15
Episode: 90100
average reward: 14.45
Episode: 90200
average reward: -35.26
Episode: 90300
average reward: -43.24
Episode: 90400
average reward: -20.49
Episode: 90500
average reward: 28.91
Episode: 90600
average reward: -25.29
Episode: 90700
average reward: -17.9
Episode: 90800
average reward: 24.04
Episode: 90900
average reward: -12.28
Episode: 91000
average reward: -63.11
Episode: 91100
average r

In [6]:
with open(weight_path, 'wb') as f:
    np.save(f, q_table)

In [7]:
import time
total_epochs, total_penalties = 0, 0
episodes = 10
total_reward = 0
for _ in range(episodes):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        state = encode(state)
        if reward == -10:
            penalties += 1
        
        epochs += 1
        total_reward += reward
#         env.render()
#         time.sleep(1/60)
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")

Results after 10 episodes:
Average timesteps per episode: 10801.0
Average penalties per episode: 108.6
Average reward per episode: 28.0
